In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models

def build_ecg_model(input_shape=(12, 5000, 1), num_classes=71):
    inputs = tf.keras.Input(shape=input_shape)

    # Temporal Convolution
    x = layers.Conv2D(filters=16, kernel_size=(1, 64), strides=(1, 8), padding='same', activation='relu')(inputs)
    x = layers.Conv2D(filters=64, kernel_size=(1, 256), strides=(1, 8), padding='same', activation='relu')(x)
    x = layers.Conv2D(filters=256, kernel_size=(1, 1024), strides=(1, 8), padding='same', activation='relu')(x)
    x = layers.Conv2D(filters=1024, kernel_size=(1, 1024), strides=(1, 8), padding='same', activation='relu')(x)
    x = layers.Conv2D(filters=1024, kernel_size=(1, 1024), strides=(1, 8), padding='same', activation='relu')(x)

    # Spatial Convolution
    x = layers.Conv2D(filters=4, kernel_size=(5, 1), padding='same', activation='relu')(x)
    x = layers.Conv2D(filters=16, kernel_size=(5, 1), padding='same', activation='relu')(x)
    
    # Feature Convolutions
    x = layers.SeparableConv2D(filters=16, kernel_size=(1, 16), depth_multiplier=1, strides=(1, 4), padding='same', activation='relu')(x)
    x = layers.SeparableConv2D(filters=32, kernel_size=(1, 16), depth_multiplier=2, strides=(1, 2), padding='same', activation='relu')(x)
    
    # Flatten and Bi-LSTM
    x = layers.Flatten()(x)
    x = layers.Bidirectional(layers.LSTM(64, return_sequences=False))(x)
    
    # Batch Normalization
    x = layers.BatchNormalization()(x)
    
    # Dense Output
    outputs = layers.Dense(num_classes, activation='sigmoid')(x)

    # Create the model
    model = models.Model(inputs, outputs)
    
    return model

# Model creation
input_shape = (12, 5000, 1)  # ECG input shape
num_classes = 71  # Number of output classes
model = build_ecg_model(input_shape, num_classes)

# Model summary
model.summary()


2024-08-29 15:07:27.629396: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-29 15:07:27.668944: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-29 15:07:27.668985: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-29 15:07:27.669023: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-29 15:07:27.676496: I tensorflow/core/platform/cpu_feature_g

ResourceExhaustedError: {{function_node __wrapped__AddV2_device_/job:localhost/replica:0/task:0/device:GPU:0}} failed to allocate memory [Op:AddV2] name: 